In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

In [2]:
####################################################
# 1. 가지 치기 없이 MNIST에 대한 모델을 훈련한다   #
####################################################

# MNIST 데이터셋 불러오기
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 각 픽셀 값이 0과 1 사이 되도록 이미지를 정규화
train_images = train_images / 255.0
test_images = test_images / 255.0

# 모델 구조 정의
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# 모델 학습
model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3057 - accuracy: 0.9135 - val_loss: 0.1303 - val_accuracy: 0.9622
Epoch 2/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1224 - accuracy: 0.9649 - val_loss: 0.0922 - val_accuracy: 0.9742
Epoch 3/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0870 - accuracy: 0.9751 - val_loss: 0.0747 - val_accuracy: 0.9812
Epoch 4/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0708 - accuracy: 0.9794 - val_loss: 0.0625 - val_accuracy: 0.9842


In [3]:
# 학습한 모델의 정확도 저장 ( 기준 정확도로 사용 )
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

# 임시 파일을 생성하고 그 파일에 학습한 모델을 저장한다.
# keras_file는 file path
_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9789999723434448
Saved baseline model to: /tmp/tmp1zonukva.h5


In [4]:
#########################################################
# 2. 가지 치기를 적용하여 사전 학습된 모델을 미세 조정  #
#########################################################
# 50%의 희소성(가중치 50%를 제로화)으로 모델을 시작하고 #
# 80%의 희소성으로 끝납니다.                            #
#########################################################

import tensorflow_model_optimization as tfmot

# 가지 치기에 필요한 API
# 사용자 정의 레이어를 잘라내거나, 내장 레이어의 일부를 수정하여 잘라냄
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# 2epoch 후 마치기 위해 end step 계산
batch_size = 128
epochs = 2
validation_split = 0.1 # 데이터셋의 10%가 validation set로 사용됨

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# 가지치기 할 모델 정의
# 희소성 50% -> 80%
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

# pruen_low_magnitude API를 사용하여 pruning
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` 재컴파일.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

Instructions for updating:
Please use `layer.add_weight` method instead.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,805
Trainable params: 20,410
Non-trainable params: 20,395
_________________________________________________________________


In [5]:
# 가지치기 적용 모델 학습 및 평가

logdir = tempfile.mkdtemp()

# 학습 과정 중 콜백을 사용해서 모델 최적 단계와 진행률, 가지치기 요악을 확인
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(), # 가지치기 wrapper를 최적기 단계로 업데이트
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir), # 진행률 추적 및 디버깅을 위한 로그를 제공
]

model_for_pruning.fit(train_images, train_labels,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 2s 4ms/step - loss: 0.0973 - accuracy: 0.9744 - val_loss: 0.1071 - val_accuracy: 0.9723
Epoch 2/2
422/422 [==============================] - 2s 4ms/step - loss: 0.1154 - accuracy: 0.9687 - val_loss: 0.0898 - val_accuracy: 0.9772


In [6]:
# pruning을 적용한 모델의 정확도 저장
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) # 기준 정확도
print('Pruned test accuracy:', model_for_pruning_accuracy) # pruning 후 정확도

# 이 예에서는 기준선에 비해 가지치기 후 테스트 정확도에 손실이 거의 없다

Baseline test accuracy: 0.9789999723434448
Pruned test accuracy: 0.9708999991416931


In [7]:
###################################
# 3. 가지치기로 적용 결과 확인    #
###################################

# 모델에서 가지치기 wrapper를 제거하는 API
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# 앞서 가지치기를 적용한 모델 저장
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmpfuo0joge.h5


In [8]:
# gzip을 통해 모델을 실제로 압축하고 압축된 크기를 측정
# 모델의 크기(바이트)를 반환한다.
def get_gzipped_model_size(file):
  
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [9]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))

Size of gzipped baseline Keras model: 78042.00 bytes
Size of gzipped pruned Keras model: 25593.00 bytes
